Universidad del Valle de Guatemala <br/>
Modelacion y Simulacion <br/>
Docente: Samuel Chavez <br/>
Semestre 2, 2020 <br/>

Dieter Esteban de Wit Torres 15146 <br/>
&nbsp;
&nbsp;

# Mini	proyecto 3
&nbsp;

In [63]:
# --- Import required libraries ---
import random
import math
import time

In [64]:
# --- Important Definitions ---
def exponential_service(lamda):
    return -float((1/lamda)*math.log(random.random()))

def event_time_persistence(s, lamda):
    return s-(1/lamda)*math.log(random.random())

In [65]:
# --- Gorila MegaComputing Simulation ---
def GorillaMegaComputing():
    rate = 40
    closing_time = 3600
    request_times = []
    requests = 0
    exits = 0
    request_time = 0
    exit_time = 0
    exit_times = [] 
    processes = 0                  
    processing_time = 0
    total_time = 0
    inf = float('inf')
    
    while True:
        if(request_time <= exit_time and request_time <= closing_time):
            time = request_time
            requests = requests + 1
            processes = processes + 1
            request_time = event_time_persistence(time, rate)
            if processes == 1:
                Y = exponential_service(rate)
                exit_time = time + Y
            request_times.append(time)
        elif(exit_time < request_time and exit_time <= closing_time):
            time = exit_time
            processes = processes - 1
            exits = exits + 1
            if processes == 0:
                exit_time = inf
            else:
                Y = exponential_service(rate)
                exit_time = time + Y
                time = exit_time
            exit_times.append(time)
        elif(min(request_time, exit_time) > closing_time and processes > 0):
            time = exit_time
            processes = processes - 1
            exits = exits + 1
            if processes > 0:
                Y = exponential_service(rate)
                exit_time = time + Y
                time = exit_time
            exit_times.append(time)
        elif(min(request_time, exit_time) > closing_time and processes==0):
            processing_time = max(time - closing_time, 0)
            break
    
    for req_time in range(len(request_times)):
        remaining_time = request_times[req_time]
        total_time = total_time + remaining_time
        
    global gorilla_requests 
    gorilla_requests = requests
    global gorilla_occupied 
    gorilla_occupied = exit_time
    global gorilla_free 
    gorilla_free = exit_time - request_time
    global gorilla_queue 
    gorilla_queue = total_time/len(request_times)
    global gorilla_prom_queue 
    gorilla_prom_queue = gorilla_queue/requests
    global gorilla_prom_queue_req 
    gorilla_prom_queue_req = requests/gorilla_queue
    global gorilla_last_req_call
    gorilla_last_req_call = request_times[-1]
    global gorilla_last_req_end
    gorilla_last_req_end = exit_times[-1]

In [66]:
def AntsSmartComputing(servers, args):
    rate = 40
    time = 0                          
    closing_time = 3600                            
    request_times = []                              
    exit_times = []                              
    processing_times = []                              
    requests = 0                                                           
    request_time = exponential_service(rate)
    exit_time = [40] * servers
    total_time = 0
    sistem_requests = [0] * (1 + servers)
    server_processes = [0] * servers
    
    while time < closing_time or sistem_requests[0] > 0:           
        search_min_exit_time = exit_time.index(min(exit_time))            
        
        if request_time < exit_time[search_min_exit_time] and request_time < closing_time:   
            time = request_time                     
            requests = requests +  1                
            request_time = request_time + exponential_service(rate)
            request_times.append(time)                  
            
            if sistem_requests[0] == 0:              
                processing_times.append(time)             
                sistem_requests[0] += 1              
                sistem_requests[1] = requests              
                exit_time[0] = time + exponential_service(rate - 30) 
                
            elif sistem_requests[0] < servers:            
                available = sistem_requests.index(0) - 1 
                processing_times.append(time)             
                sistem_requests[0] += 1              
                sistem_requests[available + 1] = requests      
                exit_time[available] = time + exponential_service(rate - 30) 
            
            else:                      
                sistem_requests[0] += 1               
        else:                          
            time = exit_time[search_min_exit_time]             
            server_processes[search_min_exit_time] += 1            
            exit_times.append(time)                 
            
            if sistem_requests[0] <= servers:            
                sistem_requests[0] -= 1              
                exit_time[search_min_exit_time] = 1000000    
                sistem_requests[search_min_exit_time + 1] = 0      
            
            else:                       
                sistem_requests[0] *= -1
                next_process = max(sistem_requests) + 1  
                sistem_requests[0] *= -1
                processing_times.append(time)            
                sistem_requests[0] -= 1              
                exit_time[search_min_exit_time] = time + exponential_service(rate - 30) 
                sistem_requests[search_min_exit_time + 1] = next_process 
                
    if args == 'requests_detailed':
        print ("Solicitudes atendididas por cada servidor de Ants SmartComputing:")
        print ("-----------------------------------------")
        for process in range (len(server_processes)):
            print ("\t" + "Servidor " + str(process + 1) + ":", server_processes[process], "solicitudes.")
        print ("-----------------------------------------")
    
    for req_time in range(len(request_times)):
        remaining_time = processing_times[req_time] - request_times[req_time]
        total_time = total_time + remaining_time
    
    global ants_requests 
    ants_requests = requests
    global ants_occupied 
    ants_occupied = exit_time
    global ants_free 
    ants_free = exit_time[0] - (exit_time[0] - request_time)
    global ants_queue 
    ants_queue = total_time
    global ants_prom_queue 
    ants_prom_queue = ants_queue/requests
    global ants_prom_queue_req 
    try:
        ants_prom_queue_req = requests/ants_queue
    except:
        ants_prom_queue_req = 0
    global ants_last_req_call
    ants_last_req_call = request_times[-1]
    global ants_last_req_end
    ants_last_req_end = exit_times[-1]

## Ejercicio	único – Servidores:

Es	usted	el/la	encargad@	de	definir	la	arquitectura	a	usar	en	el	lanzamiento	de	su	próxima	aplicación	Web:	C3 (Sistema	de	Contabilidad	del	Conde	Contador). La	junta	directiva	le	ha	solicitado	que	encuentre	el	mejor	servicio	de	hosting	para	el	proyecto.	Después	de	una	investigación	gigante,	usted	concluye	que	las	mejores	opciones	se	reducen	a	las	siguientes	dos:

### Proveedor	1	– Gorilla	Megacomputing:
    Tienen una infraestructura de servidor único, con mucha potencia de procesamiento. Ellos se enorgullecen al indicar que su servidor Enterprise puede atender hasta 100 solicitudes por segundo.

### Proveedor	2	– Ants	smart	computing:
    Tienen una infraestructura de múltiples servidores (en nube). Cada servidor es medianamente potente, y en su promoción indican que uno paga únicamente la cantidad de servidores que su aplicación requiera. Luego de su análisis de esta oferta, usted infiere que cada servidor tiene a lo sumo una décima parte de la potencia del servidor promocionado por Gorilla Megacomputing.

Las	pruebas	de	estrés	iniciales,	y	las	proyecciones	calculadas	para	los	primeros	2	años	luego	del	lanzamiento,	indican	que	su	aplicación	jamás	excederá	los	2400	solicitudes	por	minuto.	Una	auditoría	y	análisis	de	benchmark a	sistemas	similares	al	suyo,	indican	que	las	solicitudes	deberían	llegar como	un	proceso	de	Poisson,	y	que	el	tiempo	de	servicio	de	cada	solicitud	(sin	importar	la	arquitectura	de	servidor	usada)	es	modelado	adecuadamente	por	una	variable	aleatoria	exponencial.

Mañana	tiene	que	presentar	su	decisión	final	a	la	junta	directiva	del	proyecto.		Como	no	tiene	tiempo	para	hacer una	investigación	a	detalle	con	los	clientes	de	cada	proveedor,	decide	creer	en	su	promoción	y	hacer	una	simulación	para	concluir	cuál	será	la	mejor	opción.

### Task 1: 

Modele,	simule	y	analice el	comportamiento	de	ambos	sistemas	durante	una	hora	de	ejecución	de	C3,	y	para	cada	sistema	responda:

In [67]:
GorillaMegaComputing()
AntsSmartComputing(10, 'requests_detailed')

Solicitudes atendididas por cada servidor de Ants SmartComputing:
-----------------------------------------
	Servidor 1: 28986 solicitudes.
	Servidor 2: 26552 solicitudes.
	Servidor 3: 23824 solicitudes.
	Servidor 4: 20179 solicitudes.
	Servidor 5: 16183 solicitudes.
	Servidor 6: 11874 solicitudes.
	Servidor 7: 7753 solicitudes.
	Servidor 8: 4648 solicitudes.
	Servidor 9: 2551 solicitudes.
	Servidor 10: 1248 solicitudes.
-----------------------------------------


- ¿Cuántas	solicitudes atendió	cada	servidor?

In [68]:
print ("> Gorilla MegaComputing atendio", gorilla_requests, "solicitudes.")
print ("> Ants SmartComputing atendio", ants_requests, "solicitudes.")

> Gorilla MegaComputing atendio 143881 solicitudes.
> Ants SmartComputing atendio 143798 solicitudes.


- ¿Cuánto	tiempo	estuvo	cada	servidor	ocupado?

In [69]:
print ("> Gorilla MegaComputing se mantuvo ocupado por", round(gorilla_occupied, 4), "segundos.")
print ("> Ants SmartComputing se mantuvo ocupado por: \n ", ants_occupied[0], "segundos por servidor. \n ", sum(ants_occupied), "segundos totales.")

> Gorilla MegaComputing se mantuvo ocupado por 3603.5639 segundos.
> Ants SmartComputing se mantuvo ocupado por: 
  1000000 segundos por servidor. 
  10000000 segundos totales.


- ¿Cuánto	tiempo	estuvo	cada	servidor	desocupado	(idle)?

In [70]:
print ("> Gorilla MegaComputing se mantuvo desocupado por", round(gorilla_free, 4), "segundos.")
print ("> Ants SmartComputing se mantuvo desocupado por: \n ", round(ants_free/10, 4), "segundos por servidor aproximadamente. \n ", round(ants_free, 4), "segundos totales")

> Gorilla MegaComputing se mantuvo desocupado por 3.5541 segundos.
> Ants SmartComputing se mantuvo desocupado por: 
  360.0002 segundos por servidor aproximadamente. 
  3600.0017 segundos totales


- ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [71]:
print ("> Se mantuvieron en cola las solicitudes de Gorilla MegaComputing por", round(gorilla_queue, 4), "segundos.")
print ("> Se mantuvieron en cola las solicitudes de Ants SmartComputing por: \n ", round(ants_queue, 4), "segundos por servidor aproximadamente. \n ", round(ants_queue*10, 4), "segundos totales.")

> Se mantuvieron en cola las solicitudes de Gorilla MegaComputing por 1797.8621 segundos.
> Se mantuvieron en cola las solicitudes de Ants SmartComputing por: 
  19.9257 segundos por servidor aproximadamente. 
  199.257 segundos totales.


- En	promedio	¿cuánto	tiempo	estuvo	cada	solicitud en	cola?

In [72]:
print ("> En promedio, estuvo cada solicitud en cola en Gorilla MegaComputing", round(gorilla_prom_queue, 4), "segundos aproximadamente.")
print ("> En promedio, estuvo cada solicitud en cola en Ants SmartComputing", round(ants_prom_queue, 4), "segundos por servidor aproximadamente.")

> En promedio, estuvo cada solicitud en cola en Gorilla MegaComputing 0.0125 segundos aproximadamente.
> En promedio, estuvo cada solicitud en cola en Ants SmartComputing 0.0001 segundos por servidor aproximadamente.


- En	promedio,	¿cuántas	solicitudes estuvieron	en	cola cada	segundo?

In [73]:
print ("> En promedio, estuvieron", round(gorilla_prom_queue_req, 4), "solicitudes por segundo en cola en Gorilla MegaComputing aproximadamente.")
print ("> En promedio, estuvieron", round(ants_prom_queue_req, 4), "solicitudes por segundo en cola en Ants SmartComputing aproximadamente.")

> En promedio, estuvieron 80.0289 solicitudes por segundo en cola en Gorilla MegaComputing aproximadamente.
> En promedio, estuvieron 7216.7118 solicitudes por segundo en cola en Ants SmartComputing aproximadamente.


- ¿Cuál	es	el	momento	de	la	salida	de la última solicitud?

In [74]:
print ("> En Gorilla MegaComputing la ultima solicitud entro a los", round(gorilla_last_req_call, 4), "segundos y salio a los", round(gorilla_last_req_end, 4), "segundos.")
print ("> En Ants SmartComputing la ultima solicitud entro a los", round(ants_last_req_call, 4), "segundos y salio a los", round(ants_last_req_end, 4), "segundos.")

> En Gorilla MegaComputing la ultima solicitud entro a los 3599.9786 segundos y salio a los 3603.5639 segundos.
> En Ants SmartComputing la ultima solicitud entro a los 3599.9968 segundos y salio a los 3600.1397 segundos.


### Task 2: 

Determine	empíricamente	cuántos	servidores	se	necesitaría	“alquilar”	en	Ants	smart	computing	para	asegurar	que	siempre	habrá al	menos	un	servidor	disponible	para	atender	una	solicitud	dada (en	otras	palabras,	una	solicitud	nunca	tiene	que	esperar	en	cola).

In [76]:
min_servers, max_servers = 10, 1000
for server in range(min_servers, max_servers):
    AntsSmartComputing(server, '')
    if ants_queue == 0:
        print("Se requieren", server, "servidores en Ants SmartComputing para que ninguna solicitud deba esperar en cola.")
        break

Se requieren 15 servidores en Ants SmartComputing para que ninguna solicitud deba esperar en cola.


### Task 3 & 4: 

se	espera que	a	partir	del	tercer	año	del	lanzamiento	de	su	aplicación,	la	cantidad	de usuarios sufra	un	alza,	y	por	tanto	deberán	 atender como	máximo	6000	solicitudes	por	minuto.	Resuelva	el	Task1 y	el	Task2 para	esta	nueva	configuración.